<a href="https://colab.research.google.com/github/tejas2008/Sentiment-analysis/blob/master/sentiment_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)
from nltk.stem import PorterStemmer 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 
from nltk.corpus import stopwords 
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

In [26]:
len(positive_tweets)

5000

In [27]:

test_pos = positive_tweets[4000:]
train_pos = positive_tweets[:4000]
test_neg = negative_tweets[4000:]
train_neg = negative_tweets[:4000]
train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [28]:
len(train_x)

8000

In [29]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [30]:
train_y

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [31]:
stemmer = PorterStemmer()
def preprocess(tweet):
  punc = '''!()-[]{};:'"\, <>./?@#$%^&*'''
  tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
  tweet = re.sub(r'#', '', tweet)
  tokn_word = tknzr.tokenize(tweet)
  stp_words = set(stopwords.words('english')) 
  out = []
  for i in tokn_word:
    if i not in stp_words and i not in punc:
      j = stemmer.stem(i)
      out.append(j)
  return out


In [32]:
for i in range(len(train_x)):
  train_x[i] = preprocess(train_x[i])
print(train_x[0])

['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [ ]:
fre = {}
def frequency(l):
  freqs = {}
  for i in range(4000):
    for word in l[i]:
      if (word,1) in freqs:
        freqs[(word,1)] += 1
      else:
        freqs[(word,1)] = 1 
  for i in range(4000,8000):
    for word in l[i]:
      if (word,0) in freqs:
        freqs[(word,0)] += 1
      else:
        freqs[(word,0)] = 1 
  return freqs
fre = frequency(train_x)
# print(fre)

In [79]:
def vec_for_model(fre,l):
  x = []
  for i in range(len(l)):
    temp = []
    temp.append(1)
    pos_count = 0
    neg_count = 0
    for word in l[i]:
      pos_count += fre.get((word,1),0)
      neg_count += fre.get((word,0),0)
    temp.append(pos_count/500)
    temp.append(neg_count/500)
    x.append(temp)
  return x
X = np.array(vec_for_model(fre,train_x))
print(X[:10])
print(X.shape)

[[1.000e+00 6.040e+00 1.220e-01]
 [1.000e+00 7.182e+00 9.240e-01]
 [1.000e+00 6.226e+00 4.380e-01]
 [1.000e+00 5.724e+00 8.000e-03]
 [1.000e+00 6.232e+00 4.480e-01]
 [1.000e+00 5.972e+00 3.140e-01]
 [1.000e+00 8.128e+00 1.224e+00]
 [1.000e+00 6.410e+00 6.580e-01]
 [1.000e+00 1.240e+00 3.660e-01]
 [1.000e+00 5.340e-01 2.340e-01]]
(8000, 3)


In [80]:
print(train_x[0])

['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [81]:

def gradient(x,y,no_itr):
  eta = 0.001
  # wt = np.zeros((3,1))
  wt = np.random.rand(3,1)
  for i in range(no_itr):
    z = np.dot(x,wt)
    h = 1.0/(1 + np.exp(-1.0*z))
    j=(-1/len(x))*(np.dot(y.T,np.log(h))+np.dot((1-𝐲).T,np.log(1-𝐡)))
    wt=wt-(eta/len(x))*np.dot(𝐱.T,(𝐡-𝐲))
    # print('z: ',z)
    # print('h: ',h)
    # print('j: ',j)
    # print('wt: ',wt)
  return j,wt


cost,weights = gradient(X,train_y,1000)
print(cost)
print(weights)


[[0.11081009]]
[[ 0.64142319]
 [ 0.65559983]
 [-0.51786451]]


In [83]:
def pre_sentiment(tweet,weights,fre):
  l = preprocess(tweet)
  l1 = []
  l1.append(l)
  x = np.array(vec_for_model(fre,l1),dtype=np.float128)
  z = np.dot(x,weights)
  print(x)
  print(z)
  y_pr = 1/(1+np.exp(-z))
  print(y_pr)
  if y_pr >= 0.5 :
    return 'Positive'
  else:
    return 'Negative'

pre_sentiment('it is a very good movie',weights,fre)


[[1.    0.416 0.2  ]]
[[0.81057982]]
[[0.69223305]]


'Positive'